# PHẦN 1: THU THẬP DỮ LIỆU


&#9889;**Bảng phân công nhóm**

|Họ và tên|MSSV|Phân công| 
|-----|-------|------| 
|Phan Minh Triết|19120039|viết báo cáo, phần API| 
|Trần Đức Thuỵ|19120138|phần parse HTML|

## Phần Parse HTML

&#9899; **Ý tưởng:**
- Tạo cache, tạo các list thuộc tính của user
- Sử dụng SELENIUM chạy trên web soundcloud để tìm thông tin Users bằng Keyword, tạo DataFrame của Users và lưu vào File
- Tiếp tục sử dụng SELENIUM để lấy id và thông tin các track, playlist và lưu vào File

&#9899; **Nhận xét về Data thu thập được từ SoundCloud:**
- Số mẫu thu thập khoảng 2500 
- Thông tin của Playlists, Tracks gôm tên và địa chỉ url, cho thấy các playlists, tracks đều có cấu trúc tương tự nhau và phân biệt nhau bằng ID
- Thông tin của Users gôm Name, Link, Number of Followers, Number of tracks phân biệt nhau ở ID, số lượng tracks, follower phân bố không đều

*Thư viện sử dụng trong Phần 1:*

In [ ]:
import string
import requests
import requests_cache
from requests_cache import CachedSession
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from requests_html import HTMLSession

In [ ]:
#Tao cache
def check_cached(response: requests.Response) ->bool :
    if(response.text.find('"incomplete_results":true')==-1):
        return True
    else:
        return False
    
requests_cache.install_cache(expire_after = -1, filter_fn = check_cached)

In [ ]:
#Tao cac list cac thuoc tinh cua user

users_name = []

users_link = []

num_followers = []

num_tracks = []

In [ ]:

# Ham lay thong tin users bang keyword

def get_info_users(keyword):
    
    url =f'https://soundcloud.com/search/people?q={keyword}'
    
    driver.get(url)
    
    for i in range(0,50):
        
        time.sleep(1)
    
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    
    time.sleep(1)
    
    page_source = driver.page_source
    
    soup = BeautifulSoup(page_source, 'html.parser')
    
    data_list = soup.find_all('li', class_='searchList__item')
    
    for i in data_list:
        
        name = i.find('a', class_='sc-link-dark sc-link-primary')
        
        follower_track = i.find_all('span', class_='sc-visuallyhidden')
        
        if(follower_track == []):
            num_followers.append('0')
            num_tracks.append('0')
        elif (len(follower_track)==1):
            if('track' in follower_track[0]):
                num_tracks.append(follower_track[0].text)
                num_followers.append('0')
            else:
                num_followers.append(follower_track[0].text)
                num_tracks.append('0')
        else:
            num_tracks.append(follower_track[1].text)
            num_followers.append(follower_track[0].text)
            
        profile_link = name.get('href')
        
        link = 'https://soundcloud.com' + profile_link
        
        users_name.append(name.text)
        
        users_link.append(link)
        
        print(name.text, link)

In [ ]:
# Tim thong tin user bang cac keyword trong bang chu cai

driver = webdriver.Chrome()

time.sleep(5)

list_keys = list(string.ascii_lowercase)

for i in list_keys:
    
    get_info_users(i)

In [ ]:
#Tao datafame cua users va luu vao file users_data

s= {'Name':users_name, 'Link': users_link, 'Number of Followers': num_followers, 'Number of tracks': num_tracks}

df = pd.DataFrame(s)

df.to_csv('users_data.csv')

In [ ]:
#Lấy link Users 
s = pd.read_csv('users_data.csv')
users_link = s["Link"]

In [ ]:
#Khởi tạo các thuộc tính cho Playlist
playlist_name = []

playlist_link = []

added_time = []

num_like = []

tag_content = []

In [ ]:
#Hàm lấy Data Playlist bằng link của Users
def get_sets_data(user_link,driver):
    url = user_link +'/sets'
    driver.get(url)
    for i in range(0,2):
        time.sleep(1)
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')   
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data = soup.find_all('li', class_='soundList__item')
    for i in data:
        name_link = i.find('a', class_='sc-link-primary soundTitle__title sc-link-dark sc-text-h4')
        name = name_link.span.text
        link = 'https://soundcloud.com' + name_link.get('href')   
        date_time = i.find('time', class_= 'relativeTime').get('datetime')  
        date_time = date_time[:len(date_time)-5]
        date_time = date_time.replace("T", " ")
        like = i.find('button', class_='sc-button-like sc-button-secondary sc-button sc-button-small sc-button-responsive')
        like = like.text
        if('Like' in like):
            like = '0'
        tag = i.find_all('span',class_='sc-truncate sc-tagContent')
        if(tag==[]):
            tag = 'None'
        else:
            tag = tag[0].text
        playlist_name.append(name)
        playlist_link.append(link)
        added_time.append(str(date_time))
        num_like.append(like)
        tag_content.append(tag)
        print(name,link,date_time,like,tag)

In [ ]:
#Khởi chạy SELENIUM
driver = webdriver.Chrome()
time.sleep(5)
for link in users_link:
       get_sets_data(link,driver)

In [ ]:
#Lưu vào file
s = {'Playlist Name': playlist_name, 'Link': playlist_link, 'Time': added_time, 'Like': num_like, 'Hashtag': tag_content}
df_playlist = pd.DataFrame(s)
df_playlist.to_csv('playlist_data.csv', index = False)

In [ ]:
#Lấy link User
s = pd.read_csv('users_data.csv')
users_link = s["Link"]

In [ ]:
#Khởi tạo các thuộc tính track
track_name = []

track_link = []

track_added_time = []

track_num_like = []

track_tag_content = []

In [ ]:
#Hàm lấy track
def get_tracks_data(user_link,driver):
    url = user_link +'/tracks'
    driver.get(url)
    for i in range(0,2):
        time.sleep(1)
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')   
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    data = soup.find_all('li', class_='soundList__item')
    for i in data:
        name_link = i.find('a', class_='sc-link-primary soundTitle__title sc-link-dark sc-text-h4')
        if(name_link==None):
            continue
        name = name_link.span.text
        link = 'https://soundcloud.com' + name_link.get('href')   
        date_time = i.find('time', class_= 'relativeTime').get('datetime')  
        date_time = date_time[:len(date_time)-5]
        date_time = date_time.replace("T", " ")
        like = i.find('button', class_='sc-button-like sc-button-secondary sc-button sc-button-small sc-button-responsive')
        like = like.text
        if('Like' in like):
            like = '0'
        tag = i.find_all('span',class_='sc-truncate sc-tagContent')
        if(tag==[]):
            tag = 'None'
        else:
            tag = tag[0].text
        track_name.append(name)
        track_link.append(link)
        track_added_time.append(str(date_time))
        track_num_like.append(like)
        track_tag_content.append('#' + tag)
        print(name,link,date_time,like,tag)

In [ ]:
#Khởi chạy SELENIUM
driver = webdriver.Chrome()

time.sleep(5)

for link in users_link:
    get_tracks_data(link,driver)

In [ ]:
#Lưu vào file
s = {'Track Name': track_name, 'Link': track_link, 'Time': track_added_time, 'Like': track_num_like, 'Genre': track_tag_content}
df_playlist = pd.DataFrame(s)
df_playlist.to_csv('track_data.csv', index = False)

                                        -------- HẾT PHẦN COLLECT DATA --------